In [1]:
import arcpy
from arcpy.management import *
from arcpy.conversion import *
from arcpy.da import *
from arcpy.sa import *
from arcpy import env
import os
import pandas as pd
import numpy as np

In [2]:
# 读取全数据点
point_data = r"D:\ArcgisData\pred_tl\TL.gdb\HZR"

In [3]:
# 特征数据
fetatus_data = r"D:\ArcgisData\basedata\basetrain_5m.gdb"

In [4]:
# 设置工作环境
env.workspace = fetatus_data

In [8]:
# 采样
def sample_point(point_,raster_,out_name):
    Sample(raster_,point_,out_name,"NEAREST", "OBJECTID", "CURRENT_SLICE", None, '', None, None, "ROW_WISE", "TABLE")
    return None

# 导出CSV
def export_csv(table_,out_path,out_name):
    TableToTable(table_,out_path,out_name)
    return None
    

In [5]:
# 获取栅格列表
raster_list = arcpy.ListRasters()

In [6]:
table_gdb_path = r"D:\ArcgisData\pred_organic_p_n\feature_table\tableresult.gdb"
# 使用Delete_management函数删除数据库中的所有内容
arcpy.Delete_management(table_gdb_path)
# 创建一个数据库
arcpy.management.CreateFileGDB(r"D:\ArcgisData\pred_organic_p_n\feature_table", "tableresult", "CURRENT")

<Result 'D:\\ArcgisData\\pred_organic_p_n\\feature_table\\tableresult.gdb'>

In [9]:
# 逐个采样
for one_raster in raster_list:
    sample_point(point_data,one_raster,os.path.join(table_gdb_path,one_raster))

In [10]:
env.workspace = table_gdb_path

In [16]:
list_name = [_.name for _ in arcpy.ListFields(point_data)]
list_name

['OBJECTID', 'Shape', '坡位', '亚类', 'DLMC', '岩性', '属', '土种']

In [17]:
result_df = pd.DataFrame(arcpy.da.FeatureClassToNumPyArray(point_data,["OBJECTID","坡位","岩性","DLMC","亚类","属","土种"]))
result_df.rename(columns={"OBJECTID":"HZR"},inplace=True)
tables = arcpy.ListTables()
# 存储每个表的最后一个字段的数据

# 读取每个表的最后一个字段的数据
for table in tables:
    
    # 将表转换为pandas数据帧
    df = pd.DataFrame(arcpy.da.TableToNumPyArray(table, "*"))
    # 提取最后一个字段的数据
    merged_df = df[['HZR', df.columns[-1]]]
    # 合并
    result_df = pd.merge(result_df, merged_df, on=['HZR'])

In [18]:

result_df.columns

Index(['HZR', '坡位', '岩性', 'DLMC', '亚类', '属', '土种', 'TWI_5_Band_1',
       'TPI_201_Band_1', 'TPI_101_Band_1', 'TPI_11_Band_1', 'TPI_3_Band_1',
       'TMP_Band_1', 'SOILQS_Band_1', 'SLOP_Band_1', 'PRE_Band_1',
       'NIGTH_Band_1', 'NDVI_Band_1', 'DEM_Band_1', 'CUR_Band_1', 'ASP_Band_1',
       'PLCUR_Band_1', 'POCUR_Band_1', 'OSJL_Band_1', 'LAT_Band_1',
       'LON_Band_1', 'DZ_Band_1', 'DL_Band_1', 'TRI_Band_1'],
      dtype='object')

In [19]:
new_columns = [str(_).replace("_Band_1","").replace("_","").replace("DLMC","地类名称") for _ in result_df.columns]

In [20]:
result_df.rename(columns=dict(zip(result_df.columns, new_columns)),inplace=True)

In [21]:
# 存储位置
feature_out_path = r"D:\ArcgisData\pred_organic_p_n\feature_table\feature_table_result"
result_df.to_csv(os.path.join(feature_out_path,"all_tl_result.csv"))